# SAP HANA Cloud LangChain Integration with GPU Acceleration

This notebook guides you through setting up and using the SAP HANA Cloud LangChain Integration with NVIDIA GPU acceleration and TensorRT optimization.

## Authentication and Setup

This section guides you through authenticating with NVIDIA NGC and setting up the container.

### Step 1: Install and Configure NGC CLI

Download and set up the NGC CLI for authenticating with the NGC registry:

In [ ]:
!wget -O ngccli.zip https://ngc.nvidia.com/downloads/ngccli_linux.zip
!unzip -o ngccli.zip
!chmod u+x ngc-cli/ngc
# Add NGC CLI to PATH
import os
os.environ['PATH'] = os.environ['PATH'] + ':' + os.getcwd() + '/ngc-cli'

**Important**: You need to get your NGC API key from https://ngc.nvidia.com/setup/api-key

Once you have your API key, configure the NGC CLI:

In [ ]:
# Replace YOUR_API_KEY with your actual NGC API key
# !mkdir -p ~/.ngc
# !echo 'ApiKey=YOUR_API_KEY' > ~/.ngc/config

# Instead of hardcoding the API key, you can configure it interactively:
print("Please run the following command in a terminal and follow the prompts:")
print("ngc config set")

### Step 2: Log in to Docker with NGC Credentials

In [ ]:
print("Please run the following command in a terminal and follow the prompts:")
print("docker login nvcr.io")

### Step 3: Pull the Container Image

In [ ]:
!docker pull nvcr.io/plturrell/sap-enhanced/langchain-hana-gpu:latest

### Step 4: Configure SAP HANA Cloud Connection

Create a `.env` file with your SAP HANA Cloud credentials:

In [ ]:
%%writefile .env
# SAP HANA Cloud credentials
HANA_HOST=your-hana-host.hanacloud.ondemand.com
HANA_PORT=443
HANA_USER=your_username
HANA_PASSWORD=your_password

# GPU Configuration
GPU_ENABLED=true
USE_TENSORRT=true
TENSORRT_PRECISION=fp16

**Important**: Replace the placeholder values with your actual SAP HANA Cloud credentials.

### Step 5: Run the Container

Start the container with GPU support:

In [ ]:
# Run in background
!docker run -d --name langchain-hana-gpu --gpus all -p 8000:8000 \
  --env-file .env \
  nvcr.io/plturrell/sap-enhanced/langchain-hana-gpu:latest

Check if the container is running:

In [ ]:
!docker ps | grep langchain-hana-gpu

View container logs:

In [ ]:
!docker logs langchain-hana-gpu

## Testing the API

Now that the API is running, let's test its functionality.

### Check GPU Information

Get information about available GPUs:

In [ ]:
!curl http://localhost:8000/benchmark/gpu_info

### Run TensorRT Benchmark

Benchmark TensorRT vs PyTorch performance:

In [ ]:
%%writefile benchmark_request.json
{
  "model_name": "all-MiniLM-L6-v2",
  "precision": "fp16",
  "batch_sizes": [1, 8, 32, 64],
  "input_length": 128,
  "iterations": 10
}

In [ ]:
!curl -X POST "http://localhost:8000/benchmark/tensorrt" \
  -H "Content-Type: application/json" \
  -d @benchmark_request.json

### Create Vector Store

Add documents to the vector store:

In [ ]:
%%writefile add_texts_request.json
{
  "texts": [
    "NVIDIA is a technology company known for GPUs and AI.",
    "SAP HANA is an in-memory database for enterprise applications.",
    "LangChain is a framework for building LLM-powered applications.",
    "TensorRT is an SDK for high-performance deep learning inference."
  ],
  "metadatas": [
    {"source": "nvidia", "category": "company"},
    {"source": "sap", "category": "database"},
    {"source": "langchain", "category": "framework"},
    {"source": "nvidia", "category": "sdk"}
  ],
  "table_name": "DEMO_VECTORS"
}

In [ ]:
!curl -X POST "http://localhost:8000/texts" \
  -H "Content-Type: application/json" \
  -d @add_texts_request.json

### Query Vector Store

Search for similar documents:

In [ ]:
%%writefile query_request.json
{
  "query": "AI and deep learning tools",
  "k": 2,
  "table_name": "DEMO_VECTORS"
}

In [ ]:
!curl -X POST "http://localhost:8000/query" \
  -H "Content-Type: application/json" \
  -d @query_request.json

### MMR Search with GPU Acceleration

Perform a Maximal Marginal Relevance search:

In [ ]:
%%writefile mmr_request.json
{
  "query": "NVIDIA technologies",
  "k": 2,
  "fetch_k": 3,
  "lambda_mult": 0.5,
  "table_name": "DEMO_VECTORS"
}

In [ ]:
!curl -X POST "http://localhost:8000/query/mmr" \
  -H "Content-Type: application/json" \
  -d @mmr_request.json

## Cleanup

Stop and remove the container when done:

In [ ]:
!docker stop langchain-hana-gpu
!docker rm langchain-hana-gpu

## Troubleshooting

If you encounter issues with the container or GPU acceleration, try the following:

In [ ]:
# Check NVIDIA driver and CUDA installation
!nvidia-smi
!nvcc --version

In [ ]:
# Test GPU access from Docker
!docker run --gpus all --rm nvidia/cuda:11.8.0-base-ubuntu22.04 nvidia-smi

In [ ]:
# Check container logs for detailed error messages
!docker logs langchain-hana-gpu

## Learn More

For more information, refer to:
- [GitHub Repository](https://github.com/plturrell/langchain-integration-for-sap-hana-cloud)
- [VM Setup Guide](https://github.com/plturrell/langchain-integration-for-sap-hana-cloud/blob/main/docs/vm_setup_guide.md)
- [NVIDIA TensorRT Documentation](https://developer.nvidia.com/tensorrt)
- [SAP HANA Cloud Documentation](https://help.sap.com/docs/hana-cloud)